Codes needed for reading the files and preparing them for language model generation

In [11]:
import re
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

def extact_sentences(file_path):
  # pattern:  is the regular expression you have defined to extract sentences
  # file_path:path to the trascript file 

  # key: Jury value: sentence said
  dictionary_lines = {}
  with open(file_path, 'r', encoding="unicode_escape") as file:
    line = file.readline()
    while(line):
      line = re.sub(r"[\[\(].*?[\]\)]", "", line)
      line = re.sub(r'FOREMAN:', 'NO. 1:', line)
      matched_text = re.search(r'NO\.\s?(\d+):\s+(.+?)(?=NO\. \d+|$)', line)
      if matched_text is not None:
        if matched_text.group(1) in dictionary_lines:
          dictionary_lines[matched_text.group(1)].append(matched_text.group(2))
        else:
          dictionary_lines[matched_text.group(1)] = [matched_text.group(2)]
      
      line = file.readline()

  # Printing the number of sentences said by each jury (just for checking)
  for jury in dictionary_lines:
    print(jury + ":\t" + str(len(dictionary_lines[jury])))
  return dictionary_lines

def pre_process_inputs(dictionary_lines):
  # This method takes in the dictionary from previous method
  # It will return two dictionaries: one having all the tokens along
  # with their frequencies (list to counter?)
  # The other dictionary will have jury as the key (e.g., NO.2)
  # and the values are another dictionary of {token: frequency} for each jury
  
  lst_all_words = []
  dictionary_juror_words = {}
  for jury in dictionary_lines:
    # Here update all the words list and the dictionary for each jury
    lst_sentences_by_jury = dictionary_lines[jury]
    words_jorur = []
    for sentence in lst_sentences:
      ### Your code goes here (remove pass)
      pass
    dictionary_juror_words[jury] = dict(Counter(words_jorur)) 
  
  vocabulary = dict(Counter(lst_all_words))
  return vocabulary, dictionary_juror_words

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/stephenlanna/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Code to generate languge model

In [ ]:
from collections import Counter

def unigram_lm(dic_vocabulary, dic_juror_words):
  # This method will generate the unigram language model for each juror
  # The return value is a dict with juror as key and value as his language model
  # the inputs are the two dictionaries from pre_process_inputs method
  dic_unigram_lm = {}

  for jury in dic_juror_words:
    dic_words_frequency = dic_juror_words[jury]
    language_model = {key: 0 for key in dic_vocabulary}
    # Here you will calculate the probabilities with MLE and add-one smoothing
    for key in language_model:
      ### Remove pass
      pass
  return dic_unigram_lm

Code to generate T-SNE plot

In [ ]:
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt
import numpy as np


def tsne_generator(dic_unigram_lm):
  tsne = TSNE(n_components=2)
  lst_freq = []
  lst_names = []
  for juror in dic_unigram_lm:
    lst_names.append(juror)
    
    # frequencies is a dictionary of token: frequencies
    # sort this by key and then convert values to the list and append it 
    # to the lst_freq
    frequencies = dic_unigram_lm[juror]
    ##### 
    # Your code goes here and then remove freq=[0] and instead of [0]
    # save your results in freq
    freq = [0]
    lst_freq.append(freq)
  
  # Convert the list to a NumPy array
  np_array = np.array(lst_freq)
  data = np_array

  # getting vectors with tsne
  vectors = tsne.fit_transform(data)

  # Your code for plot goes here
  # fig, ax = plt.subplots()
  # Some codes here to define what to be shown
  ####
  # making sure the legend is shown (uncomment)
  #ax.legend(bbox_to_anchor=(1.1, 1.05))
  #ax.grid(True)
  #plt.show()

In [12]:
path = r"/Users/stephenlanna/Documents/NLP/12AngryMen (1).txt"

dictionary_lines = extact_sentences(path)

print(dictionary_lines)

8:	93
1:	52
2:	23
3:	88
4:	33
5:	29
6:	19
7:	40
9:	20
11:	22
12:	13
10:	34
{'8': ['And it might be helpful to fix your mind the equation: 7+3=10. For jurors 7, 3, and 10 also play extremely important parts.', 'A quiet, thoughtful, gentle man. A man who sees all sides of every question and constantly seeks the truth. A man of strength tempered with compassion. Above all, he is a man who wants justice to be done and will fight to see that it is.', "Oh. I'm sorry. ", ". I don't know.", "He's nineteen years old.", 'No.', "I don't know whether I believe it or not. Maybe I don't.", "There were eleven votes for guilty. It's not so easy for me to raise my hand and send a boy off to die without talking about it first.", 'No one.', "I don't want to change your mind. I just want to talk for a while. Look, this boy's been kicked around all his life. You know, living in a slum, his mother dead since he was nine. That's not a very good head start. He's a tough, angry kid. You know why slum kids get 